In [8]:

from collections import Counter
import numpy as np
from numpy import genfromtxt
import scipy.io
from scipy import stats
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import cross_validate
import sklearn.tree
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

import random

In [9]:
path_train = 'datasets/titanic/titanic_training.csv'
data = genfromtxt(path_train, delimiter=",", dtype=None)
path_test = 'datasets/titanic/titanic_testing_data.csv'
test_data = genfromtxt(path_test, delimiter=",", dtype=None)
y = data[1:, 0]  # label = survived
class_names = ["Died", "Survived"]
data = np.delete(data,np.where(data[0,:] == b"ticket"), axis=1)
num_features = data.shape[1]

<ipython-input-9-5f191a484ea9>:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  data = genfromtxt(path_train, delimiter=",", dtype=None)
<ipython-input-9-5f191a484ea9>:4: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  test_data = genfromtxt(path_test, delimiter=",", dtype=None)


In [10]:
for i in range(num_features):
    if any(data[1:, i] == b""):
        column = data[1:, i]
        print(Counter(column).most_common(2))
        if Counter(column).most_common(1)[0][0] == b"":
            data[1:,:][column == b"", i] = Counter(column).most_common(2)[1][0]
        else:
            data[1:,:][column == b"", i] = Counter(column).most_common(1)[0][0]

df = pd.DataFrame(data)
df


[(b'0', 613), (b'1', 386)]
[(b'3', 540), (b'1', 253)]
[(b'male', 650), (b'female', 349)]
[(b'', 196), (b'22', 35)]
[(b'0', 670), (b'1', 262)]
[(b'0', 751), (b'1', 138)]
[(b'8.05', 51), (b'7.75', 43)]
[(b'', 773), (b'C23 C25 C27', 6)]
[(b'S', 711), (b'C', 195)]


,0,1,2,3,4,5,6,7,8
0,b'survived',b'pclass',b'sex',b'age',b'sibsp',b'parch',b'fare',b'cabin',b'embarked'
1,b'0',b'3',b'male',b'22',b'0',b'0',b'8.05',b'C23 C25 C27',b'S'
2,b'0',b'1',b'male',b'22',b'0',b'0',b'135.6333',b'C23 C25 C27',b'C'
3,b'0',b'2',b'male',b'23',b'0',b'0',b'15.0458',b'C23 C25 C27',b'C'
4,b'0',b'2',b'male',b'42',b'0',b'0',b'13',b'C23 C25 C27',b'S'
...,...,...,...,...,...,...,...,...,...
996,b'0',b'3',b'male',b'16',b'0',b'0',b'7.775',b'C23 C25 C27',b'S'
997,b'1',b'3',b'male',b'22',b'0',b'0',b'7.225',b'C23 C25 C27',b'C'
998,b'0',b'2',b'male',b'63',b'1',b'0',b'26',b'C23 C25 C27',b'S'
999,b'0',b'3',b'female',b'41',b'0',b'5',b'39.6875',b'C23 C25 C27',b'S'


In [33]:
import csv
file = csv.DictReader(open(path_train))
dict_filter = lambda x, y: dict((i, x[i]) for i in x if i in set(y))
keys_wanted = ("sex", "fare", "cabin", "embarked")
file_dict = [dict_filter(row, keys_wanted) for row in file]
V = DictVectorizer()
one_hot_data = V.fit_transform(file_dict)
feature = V.get_feature_names()
one_hot_data.toarray().shape

(1000, 411)

In [29]:
data[1:, [0,2,3,4,5]].shape

(1000, 5)

In [36]:
new_data = np.hstack((data[1:, [0,2,3,4,5]], one_hot_data.toarray()))
new_data.shape

(1000, 416)

In [37]:
feature

['cabin=',
 'cabin=A10',
 'cabin=A11',
 'cabin=A16',
 'cabin=A19',
 'cabin=A21',
 'cabin=A23',
 'cabin=A24',
 'cabin=A26',
 'cabin=A31',
 'cabin=A32',
 'cabin=A34',
 'cabin=A36',
 'cabin=A5',
 'cabin=A7',
 'cabin=B10',
 'cabin=B101',
 'cabin=B102',
 'cabin=B11',
 'cabin=B19',
 'cabin=B20',
 'cabin=B22',
 'cabin=B24',
 'cabin=B26',
 'cabin=B28',
 'cabin=B3',
 'cabin=B30',
 'cabin=B36',
 'cabin=B38',
 'cabin=B4',
 'cabin=B41',
 'cabin=B42',
 'cabin=B45',
 'cabin=B49',
 'cabin=B5',
 'cabin=B50',
 'cabin=B51 B53 B55',
 'cabin=B52 B54 B56',
 'cabin=B57 B59 B63 B66',
 'cabin=B58 B60',
 'cabin=B61',
 'cabin=B69',
 'cabin=B71',
 'cabin=B73',
 'cabin=B77',
 'cabin=B78',
 'cabin=B82 B84',
 'cabin=B96 B98',
 'cabin=C101',
 'cabin=C103',
 'cabin=C104',
 'cabin=C105',
 'cabin=C106',
 'cabin=C110',
 'cabin=C116',
 'cabin=C118',
 'cabin=C123',
 'cabin=C124',
 'cabin=C125',
 'cabin=C126',
 'cabin=C130',
 'cabin=C132',
 'cabin=C2',
 'cabin=C22 C26',
 'cabin=C23 C25 C27',
 'cabin=C28',
 'cabin=C30',
 'c

In [41]:
new_features = np.hstack((data[0,:], feature))
new_features.shape

(420,)

In [42]:
one_hot_data.toarray().shape

(1000, 411)

In [44]:
len(feature)

411

In [47]:
new_data[0,0].astype("float")

0.0

In [56]:
array = data[1:,5].astype("float")

In [57]:
array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 2., 0., 2., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 2., 2., 2., 0., 0., 2., 0.,
       0., 0., 1., 0., 1., 2., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 2.,
       0., 2., 2., 0., 0., 2., 0., 0., 0., 3., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 2., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       2., 0., 1., 2., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 2., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 2., 2., 1., 0., 5., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 2., 0., 1., 2., 1., 2., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 1.